# Group Counterfactual Explanation in Recommender Systems - Sliding Window Approach by Stratigi's (2025)

## 0. Imports

In [1]:
import os
import pandas as pd
import numpy as np

from recoxplainer.config import cfg
from recoxplainer.data_reader.data_reader import DataReader
from recoxplainer.models import ALS
from recoxplainer.evaluator import Splitter, Evaluator
from recoxplainer.recommender.grouprecommender import GroupRecommender


## 1. Load and Prepare Data


In [2]:
data = DataReader(**cfg.data.ml100k)

# Re-arrange users' and items' Ids
data.make_consecutive_ids_in_dataset()

# Because ALS works on implicit feedback we need to binarize it:+
data.binarize(binary_threshold=1)

# Prepare train and test sets:
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

# Prepare groups and movie IDs
movie_ids = data.dataset["itemId"].unique()
all_groups = data.read_groups('groupsWithHighRatings5')

c:\Users\usuar\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\Users\usuar\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## 2. Process each group

In [5]:
# for group_id in all_groups:
#     group_id = group_id.strip('\n')
#     members = data.parse_group_members(group_id)
#     print(f"Processing group: {members}")
    
#     ## Get candidate movies for the recommendation - not seen by the group
#     candidate_movies = data.get_items_for_group_recommendation(data.dataset, movie_ids, members)


#     ## Train model
#     als = ALS(**cfg.model.als)
#     als.fit(train)

#     # 4. Generate Recommendations
#     recommender = Recommender(train, als)
#     target_item = recommender.recommend_group(members, candidate_movies, 0)





for group_id in all_groups:
    group_id = group_id.strip('\n')
    members = data.parse_group_members(group_id)
    print(f"Processing group: {members}")
    
    # Get candidate movies not seen by the group
    candidate_movies = data.get_items_for_group_recommendation(
        data.dataset, 
        movie_ids,
        members
    )

    # Train model
    als = ALS(**cfg.model.als)
    als.fit(train)

    # Create recommender and get recommendations
    recommender = GroupRecommender(train, als)
    recommendations = recommender.recommend_group_unseen_items(
        members, 
        candidate_movies.tolist(), 
        top_k=0  # Set to 0 for just the top item, or any positive integer for top-k
    )

    print(f"Recommendations: {recommendations}")


  0%|          | 0/10 [00:00<?, ?it/s]

Processing group: [522, 385, 234, 452, 594]


  0%|          | 0/10 [00:00<?, ?it/s]

Recommendations: 10
Processing group: [522, 385, 234, 246, 428]


  0%|          | 0/10 [00:00<?, ?it/s]

...

Recommendations: 83
Processing group: [14, 156, 517, 462, 448]


  0%|          | 0/10 [00:00<?, ?it/s]

Recommendations: 239
Processing group: [14, 156, 517, 89, 28]


100%|██████████| 10/10 [00:00<00:00, 11.73it/s]


Recommendations: 364


In [4]:
# import operator
# from typing import Dict, List, Union
# import pandas as pd
# import numpy as np

# class GroupRecommender(GenericRecommender):
#     """
#     Extension of GenericRecommender to support group recommendations.
#     """

#     def __init__(self, dataset_metadata, model, top_n: int = 10):
#         super(GroupRecommender, self).__init__(dataset_metadata, model, top_n)
    
#     def generate_recommendation(self, user_id: int, movie_ids_to_pred: List[int]) -> Dict[int, float]:
#         """
#         Generate predictions for a single user for a list of items.
#         This mirrors the original generate_recommendation function.
        
#         Args:
#             user_id: User ID
#             movie_ids_to_pred: List of movie IDs to predict ratings for
            
#         Returns:
#             Dictionary mapping movie IDs to predicted ratings, sorted by prediction value
#         """
#         # Get predictions from the model
#         pred_ratings = self.model.predict(user_id, movie_ids_to_pred)
        
#         # Sort predictions in descending order
#         index_max = (-pred_ratings).argsort()[:]
        
#         # Create dictionary with sorted predictions
#         predictions = {}
#         for i in index_max:
#             predictions[movie_ids_to_pred[i]] = float(pred_ratings[i])
        
#         return predictions
    
#     def recommend_group(self, 
#                        members: List[int], 
#                        candidate_movies: List[int],
#                        aggregation_method: str = "average") -> Dict[int, float]:
#         """
#         Generate recommendations for a group of users.
        
#         Args:
#             members: List of user IDs in the group
#             candidate_movies: List of movie IDs not seen by any group member
#             aggregation_method: Method to aggregate individual preferences
            
#         Returns:
#             Dictionary mapping movie IDs to aggregated prediction scores
#         """
#         # Get predictions for each group member
#         predictions = {}
#         for member in members:
#             user_pred = self.generate_recommendation(member, candidate_movies)
#             predictions[member] = user_pred
        
#         # Aggregate predictions
#         return self._aggregate_predictions(predictions, len(members), aggregation_method)
    
#     def _aggregate_predictions(self, 
#                               predictions: Dict[int, Dict[int, float]], 
#                               group_size: int,
#                               method: str = "average") -> Dict[int, float]:
#         """
#         Aggregate predictions for a group.
        
#         Args:
#             predictions: Dictionary mapping user IDs to their predictions
#             group_size: Size of the group
#             method: Aggregation method
            
#         Returns:
#             Dictionary mapping item IDs to aggregated scores
#         """
#         scores = {}
        
#         # Sum up scores for each item across all users
#         for user, pred in predictions.items():
#             for m in pred:
#                 if m in scores:
#                     scores[m] = scores[m] + pred[m]
#                 else:
#                     scores[m] = pred[m]
        
#         # Apply aggregation method
#         if method == "average":
#             # Average strategy (this matches your original code)
#             group_pred = {}
#             for m in scores:
#                 group_pred[m] = scores[m] / group_size
                
#         elif method == "least_misery":
#             # Least misery strategy
#             group_pred = {}
#             for m in scores.keys():
#                 min_score = float('inf')
#                 for user, preds in predictions.items():
#                     if m in preds and preds[m] < min_score:
#                         min_score = preds[m]
#                 if min_score != float('inf'):
#                     group_pred[m] = min_score
                    
#         elif method == "most_pleasure":
#             # Most pleasure strategy
#             group_pred = {}
#             for m in scores.keys():
#                 max_score = float('-inf')
#                 for user, preds in predictions.items():
#                     if m in preds and preds[m] > max_score:
#                         max_score = preds[m]
#                 if max_score != float('-inf'):
#                     group_pred[m] = max_score
#         else:
#             # Default to average if method not recognized
#             group_pred = {}
#             for m in scores:
#                 group_pred[m] = scores[m] / group_size
        
#         # Sort predictions (highest first)
#         sorted_pred = dict(sorted(group_pred.items(), key=operator.itemgetter(1), reverse=True))
        
#         return sorted_pred
    
#     def recommend_group_unseen_items(self,
#                                    group: List[int],
#                                    item_ids: List[int],
#                                    top_k: int = 0) -> Union[int, List[int]]:
#         """
#         Generate recommendations of unseen items for a group.
        
#         Args:
#             group: List of group member IDs
#             item_ids: List of item IDs not seen by the group
#             top_k: Number of recommendations to return (0 for just top-1, negative for all)
            
#         Returns:
#             Single item ID or list of item IDs
#         """
#         sorted_pred = self.recommend_group(group, item_ids)
        
#         if top_k == 0:
#             # Return just the top item
#             return next(iter(sorted_pred))
#         elif top_k < 0:
#             # Return all items
#             return list(sorted_pred.keys())
#         else:
#             # Return top-k items
#             list_rec = []
#             i = 0
#             for key in sorted_pred:
#                 list_rec.append(key)
#                 i = i + 1
#                 if i == top_k:
#                     break
#             return list_rec